<a href="https://colab.research.google.com/github/ghadabenamira/IT300-BI-SunWise-Analytics/blob/main/ETL_Solar_Weather_IT300.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

df = pd.read_csv("solar_weather.csv")
df.head()

df.info()
df.describe()

#Column names were standardized by converting them to lowercase and
#removing spaces and special characters to improve readability
#and prevent processing errors.
df.columns = (
    df.columns
    .str.strip()
    .str.lower()
    .str.replace(" ", "_")
    .str.replace("[", "")
    .str.replace("]", "")
    .str.replace("/", "_")
)

#print the new columns after transformation.
print(df.columns.tolist())

#The time column was converted to datetime format,
# and additional features such as year and day were extracted to support time-based analysis.
df['time'] = pd.to_datetime(df['time'])
df['year'] = df['time'].dt.year
df['day'] = df['time'].dt.day

#there's no missing values in our dataset; however, we want to be sure.
df.isna().sum()

#Columns containing only zero values were removed
#as they do not contribute meaningful information to the analysis.
df = df.drop(columns=df.columns[(df == 0).all()])


#Outliers in energy production were removed using the Interquartile Range (IQR)
# method to reduce the impact of extreme values.
Q1 = df['energy_deltawh'].quantile(0.25)
Q3 = df['energy_deltawh'].quantile(0.75)
IQR = Q3 - Q1

df = df[
    (df['energy_deltawh'] >= Q1 - 1.5 * IQR) &
    (df['energy_deltawh'] <= Q3 + 1.5 * IQR)
]

#Cloud coverage was grouped into low, medium, and high
#categories using bins to simplify weather condition analysis.
df['cloud_level'] = pd.cut(
    df['clouds_all'],
    bins=[0, 30, 70, 100],
    labels=['Low', 'Medium', 'High']
)


# Numerical features were normalized using Min-Max scaling to bring them to a
# common range between 0 and 1, ensuring consistency and comparability across
# variables.
scaler = MinMaxScaler()
df[['ghi', 'temp', 'wind_speed']] = scaler.fit_transform(
    df[['ghi', 'temp', 'wind_speed']]
)



# Validate expected columns
expected_columns = ['time', 'ghi', 'temp', 'wind_speed', 'energy_deltawh', 'clouds_all']
missing_cols = [col for col in expected_columns if col not in df.columns]

if missing_cols:
    print("Missing expected columns:", missing_cols)
else:
    print("All expected columns are present.")



df.to_csv("solar_weather_clean.csv", index=False)

print(df.head())  # Shows the first 5 rows by default
print(df.head(10))  # Shows the first 10 rows
print(df.tail())  # Last 5 rows
print(df.shape)  # Shows (rows, columns)






















<class 'pandas.core.frame.DataFrame'>
RangeIndex: 196776 entries, 0 to 196775
Data columns (total 17 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   Time                    196776 non-null  object 
 1   Energy delta[Wh]        196776 non-null  int64  
 2   GHI                     196776 non-null  float64
 3   temp                    196776 non-null  float64
 4   pressure                196776 non-null  int64  
 5   humidity                196776 non-null  int64  
 6   wind_speed              196776 non-null  float64
 7   rain_1h                 196776 non-null  float64
 8   snow_1h                 196776 non-null  float64
 9   clouds_all              196776 non-null  int64  
 10  isSun                   196776 non-null  int64  
 11  sunlightTime            196776 non-null  int64  
 12  dayLength               196776 non-null  int64  
 13  SunlightTime/daylength  196776 non-null  float64
 14  weather_type        